<a href="https://colab.research.google.com/github/av-gav/itmo-nlp/blob/main/Task2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# task 2 for existing model and fine tuning

In [1]:
!pip install datasets evaluate transformers[sentencepiece] accelerate pycountry

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 125.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!unzip drive/MyDrive/train/lid.zip

Mounted at /content/drive
Archive:  drive/MyDrive/train/lid.zip
   creating: lid/
   creating: lid/train/
  inflating: lid/train/data-00000-of-00001.arrow  
  inflating: lid/train/state.json    
  inflating: lid/train/cache-934fab53fb367c30.arrow  
  inflating: lid/train/dataset_info.json  
  inflating: lid/train/cache-c231bff8c615096d.arrow  
  inflating: lid/train/cache-a0c1297d3543dfd3.arrow  
   creating: lid/test/
  inflating: lid/test/data-00000-of-00001.arrow  
  inflating: lid/test/state.json     
  inflating: lid/test/cache-6df024453add90c3.arrow  
  inflating: lid/test/dataset_info.json  
  inflating: lid/test/cache-7a70d1f4e2e9c7fc.arrow  
  inflating: lid/test/cache-ed04687f80a001d5.arrow  
  inflating: lid/test/cache-3acb765c1f1282d9.arrow  
  inflating: lid/test/cache-9869a57e9096f4ef.arrow  
  inflating: lid/test/cache-2fb02146d694e746.arrow  
 extracting: lid/dataset_dict.json   


In [2]:
import pycountry
import datasets

wili = datasets.load_dataset('wili_2018')
# names for languages not in iso-639-3 from wikipedia
non_iso_languages = {'roa-tara': 'Tarantino', 'zh-yue': 'Cantonese', 'map-bms': 'Banyumasan',
                      'nds-nl': 'Dutch Low Saxon', 'be-tarask': 'Belarusian-Tarask'}

diff = {'Russia Buriat':'Buriat', 'Kirghiz':'Kyrgyz', 'Ossetian':'Ossetic'}
# create dictionary from data set labels to language names
lab2lang = {}
for i, lang in enumerate(wili['train'].features['label'].names):
  full_lang = pycountry.languages.get(alpha_3=lang)
  if full_lang:
    lab2lang[i] = diff.get(full_lang.name, full_lang.name)
  else:
    lab2lang[i] = non_iso_languages[lang]

Generating train split:   0%|          | 0/117500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/117500 [00:00<?, ? examples/s]

Dataset wili_2018 downloaded and prepared to /root/.cache/huggingface/datasets/wili_2018/WiLI-2018 dataset/1.1.0/78d7fe4a9d0a01168e45657f302c776ee0afc0978d44e2c3759f4c4975b845f5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from datasets import ClassLabel
ds = datasets.load_from_disk('lid')

features = ds['train'].features.copy()
class_label = features['label']
langs = class_label.names
id2label = {idx: langs[idx] for idx in range(len(langs))}
label2id = {v: k for k,v in id2label.items()}
common = [label2id[x] for x in langs if x in lab2lang.values()]
features['label'] = ClassLabel(names=[lab2lang[k] for k in sorted(lab2lang)])
mapper = {label2id[v]:k for k,v in lab2lang.items() if v in label2id}
def map_lang(x):
  x['label'] = mapper[x['label']]
  return x
ds_part = ds.filter(lambda x: x['label'] in common).cast(features).map(map_lang)

Filter:   0%|          | 0/41594 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10399 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/38276 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/9568 [00:00<?, ? examples/s]

Map:   0%|          | 0/38276 [00:00<?, ? examples/s]

Map:   0%|          | 0/9568 [00:00<?, ? examples/s]

In [4]:
n_labs = len(lab2lang)
id2lab = {i:f'LAB_{i}' for i in range(n_labs)}
lab2id = {v:k for k,v in id2lab.items()}

In [5]:
from transformers import CanineTokenizer, AutoModelForSequenceClassification, pipeline

tokenizer = CanineTokenizer.from_pretrained("google/canine-c", padding="longest")

model = AutoModelForSequenceClassification.from_pretrained("SebOchs/canine-c-lang-id",
                                                             num_labels=n_labs, id2label=id2lab, label2id=lab2id)

pipe = pipeline("text-classification",
                tokenizer = tokenizer,
                model = model,
                devoce = 0)

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [53]:
import evaluate

metric = evaluate.load('accuracy')
evaluator = evaluate.evaluator("text-classification")

result = evaluator.compute(model_or_pipeline=pipe, 
                           data=ds_part['test'].select(range(1000)), metric=metric, label_mapping=lab2id)

print(result)


{'accuracy': 0.794, 'total_time_in_seconds': 271.81565637399945, 'samples_per_second': 3.6789639468893194, 'latency_in_seconds': 0.27181565637399946}


In [6]:
from transformers import DataCollatorWithPadding

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding='max_length')

tokenized_ds = ds.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

n_labs = len(langs)
model = AutoModelForSequenceClassification.from_pretrained("SebOchs/canine-c-lang-id",
                                                           ignore_mismatched_sizes=True,
                                                           num_labels=n_labs, 
                                                           id2label=id2label, label2id=label2id)

Map:   0%|          | 0/41594 [00:00<?, ? examples/s]

Map:   0%|          | 0/10399 [00:00<?, ? examples/s]

Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at SebOchs/canine-c-lang-id and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([235, 768]) in the checkpoint and torch.Size([36, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([235]) in the checkpoint and torch.Size([36]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    "train_out",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True,
    optim="adafactor",
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_ds["train"].shuffle().select(range(3600)),
    eval_dataset=tokenized_ds["test"].select(range(720)),
    data_collator=data_collator,
    tokenizer=tokenizer,
)


In [10]:
trainer.train()

Step,Training Loss
500,1.757500
1000,0.992700
1500,0.689600
2000,0.546100


Step,Training Loss
500,1.757500
1000,0.992700
1500,0.689600
2000,0.546100
2500,0.365300


TrainOutput(global_step=2700, training_loss=0.8343346009431062, metrics={'train_runtime': 1625.5942, 'train_samples_per_second': 6.644, 'train_steps_per_second': 1.661, 'total_flos': 1.41910569713664e+16, 'train_loss': 0.8343346009431062, 'epoch': 3.0})

In [14]:
import evaluate

metric = evaluate.load('accuracy')
evaluator = evaluate.evaluator("text-classification")

pipe = pipeline("text-classification",
                tokenizer = tokenizer,
                model = model,
                device = "cuda:0")

result = evaluator.compute(model_or_pipeline=pipe, 
                           data=ds['test'].select(range(720, 1440)), 
                           metric=metric, 
                           label_mapping=label2id)

print(result)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


{'accuracy': 0.8458333333333333, 'total_time_in_seconds': 19.64494798900023, 'samples_per_second': 36.65064424722064, 'latency_in_seconds': 0.027284649984722544}


In [15]:
trainer.save_model('model')

In [ ]:
trainer.train()

Step,Training Loss


Step,Training Loss
